# Airtravel Optimization Program

The below code is a our implementation of a program that calculates the lowest cost route for a aircraft to take. The user provides a csv file, that is read, line by line (containing the airports needed to go and aircrft used) and determines the optimal route to take in order to minimise the costs. 

The base currency is euro. In order to refuel the aircraft, you must purchase the refuel the aircraft to full using the local currecy. One kilometer requires one unit of fuel and can be purchased with one unit of the local currency. This means that the gain and loss of the cost of fuel is in the conversion rate between euro and the local currency.

To complete this exercise, we have implemented various data structres and algorithms. These will be explained as we make our way down the notebook.

In [1]:
import csv
import os
from math import pi, sin, cos, acos, floor

## Preparing:

### Files will be read in and  each line stored as a a list 

In [2]:
def read_file(file_name):
    """Read data in the file and Create Matrix"""
    datamatrix = []
    datafile = open(file_name)
    for line in datafile:
        datamatrix.append(line.split(","))
    datafile.close()
    return datamatrix

## Building the classes:

The Aircraft and Airport classes are defined here. Each class contains getters and setters to allow for access to data and changing of data. 

In [3]:
class Aircraft: # Class that contains all aircraft
    
    def __init__(self, model, manufacturer, max_capacity, current_capacity = 0):
        self._model = model # model of the aircraft
        self._manufacturer = manufacturer # manufacturer of the aircraft
        self._max_capacity = max_capacity # max capacity of the aircraft
        self._current_capacity = current_capacity # current capacity of the aircraft
    
    @property
    def max_capacity(self):
        return self._max_capacity # returns the max distance
    @property
    def manufacturer(self):
        return self._manufacturer # returns the manufacturer
    @property
    def model(self):
        return self._model # returns the model
    @property
    def current_capacity(self):
        return self._current_capacity # returns the current capacity left in the plane

class Airport:
    
    def __init__(self, airport_code, longitude, latitude, exchangeRate):
        self.airport_code = airport_code # airport code of the airport
        self._longitude = longitude # longitude of the airport
        self._latitude = latitude # latitude of the airport
        self._exchange_rate = exchangeRate # currency used by the airport
        
    @property
    def longitude(self):
        return self._longitude # returns the longitude of the airport
    @property
    def latitude(self):
        return self._latitude # returns the latitude of the airport
    @property
    def exchange_rate(self):
        return self._exchange_rate # returns the currency used by the airport
    @exchange_rate.setter
    def exchange_rate(self, exchangeRate):
        self._exchange_rate = exchangeRate # sets the currency of the airport

## Building the Aircraft

Below is a function that builds each of the aircraft. The time complexity is O(n), and a dictionary of aircraft objects is returned at the end. Only one aircraft object is instantiated to avoid instantiating an unecessary amount of objects. 

In [4]:
def buildAircraft(plane):
    """Builds objects for each of the aircraft - with attributes model, manufacturer, and range.
    Returns a dictionary of this"""
    aircraftDict = {}
    with open('aircraft.csv', newline='', encoding="utf8") as airplane_file:  # opens the csv file
        reader = csv.reader(airplane_file)  # reads the cotents to a variable
        next(reader, None)  # returns none at the end of the file
        for airplane in reader:  # iterates through the reader
            if airplane[0] == plane:
                if airplane[2] == "imperial":
                    airRange = int(airplane[4]) * 1.609
                else:
                    airRange = airplane[4]
                aircraftDict[airplane[0]] = Aircraft(airplane[0], airplane[3], airRange)
    if len(aircraftDict) == 0:
        return False
    else:
        return aircraftDict

## Building Airport Objects

Below is a function that builds each of the airports. The time complexity is O(n^2), and a dictionary of airport objects is returned at the end. The function looks through all three CSV files to build an Airport object for each airport that contains the name of the airport, its longitude and latitude, and the exchange rate of the country it is in.

In [5]:
def buildAirports(listy):
    """Takes in a list of the route to be analysed. The airports are searched for in the csv.
    Objects of that airport are then created - with attributes latitude, longitude, and the exchange rate to Euro.
    It returns a dictionary with airport_code as key and object as value"""
    listx = listy[:-1]
    airport_list = []  # creates a new list

    with open('airport.csv', newline='', encoding="utf8") as airport_file:  # opens the csv file
        reader = csv.reader(airport_file)  # reads the cotents to a variable
        next(reader, None)  # returns none at the end of the file
        for airport in reader:  # iterates through the reader
            if airport[4] in listx:
                airport_code = airport[4]  # assigns variable
                country_name = airport[3]  # assigns variable
                longitude = airport[6]  # assigns variable
                latitude = airport[7]  # assigns variable
                templist = [airport_code, country_name, longitude, latitude]
                airport_list.append(templist)

    country_currency_list = []  # creates a new list

    with open('countrycurrency.csv', newline='', encoding="utf8") as countrycurrency_file:  # opens the csv file
        reader = csv.reader(countrycurrency_file)  # reads the cotents to a variable
        next(reader, None)  # returns none at the end of the file
        for country in reader:  # iterates through the reader
            temp_list = [country[0], country[14]]  # temp list created
            country_currency_list.append(temp_list)  # appends temp list to the main list

    currency_list = []  # creates a new list

    with open('currencyrates.csv', newline='', encoding="utf8") as currencyrates_file:  # opens the csv file
        reader = csv.reader(currencyrates_file)  # reads the cotents to a variable
        next(reader, None)  # returns none at the end of the file
        for currency in reader:  # iterates through the reader
            temp_list = [currency[1],currency[2]]  # temp list created
            currency_list.append(temp_list)  # appends temp list to the main list

    #Outer for loops goes through list of countries and the currency they have. Inner loop will go through currency 
    #and exchange rate list, matches currency to exchange rate, and creates a final list of lists with the 
    #country and currency rate in each inner list.
    final_list = []
    for i in country_currency_list:
        for x in currency_list:
            if i[1] == x[0]:
                templist = [i[0], x[1]]
                final_list.append(templist)

    #Outer for loop will go through list of lists that contains airport, country, latitude, longitude, 
    #and the inner loop will go through the list of airports and currency information and then match 
    #them with the airport in the outer list. The inner loop will then extend 
    x = 0
    i = 0 
    while x < len(airport_list):
        while i < len(final_list):
            if airport_list[x][1] == final_list[i][0]:
                airport_list[x].extend(final_list[i])
                break
            i+=1
        x+=1
    # Make a dictionary with the Airport code as the key and the value being the airport object of that key
    finalAirports = {}
    for i in airport_list:
        finalAirports[i[0]] = Airport(i[0], i[2], i[3], i[5])

    return finalAirports

## Distance

## Calculating all Route Permutations

This recursive function is our implementation of a permutation method. 

In [6]:
def permutation(lst): 
    if len(lst) == 0: 
        return [] 
    if len(lst) == 1: 
        return [lst] 
    l = [] 
    for i in range(len(lst)): 
        m = lst[i] 
        remLst = lst[:i] + lst[i+1:] 
        for p in permutation(remLst): 
            l.append([m] + p) 
    return l 

## Caclulating all possible routes

This function takes in a list of airports and will calculate all possible routes possible with that set of airports. The complexity of O(n)

In [7]:
def allPerms(listx):
    """ Creates permutations of all possible routes using the input list of airports and plane """
    
    a = listx[0] # takes the departure airport
    permutation_list = listx[1:-1] # creates a list that removes the departure airport and the aircraft

    count = 0
    permlist = []
    newpermlist = []
    perm = permutation(permutation_list) 
    for i in list(perm): 
        permlist.append(i)

    for perms in permlist:
        perms = [a] + list(perms) + [a]
        newpermlist.append(perms)

    return newpermlist


## Great circle calculation for distances between two (longitude, latitude) coordinates.

In [8]:
def distanceBetweenAirports(latitude1, longitude1, latitude2, longitude2):
    radius_earth = 6371  # km
    theta1 = longitude1 * (2 * pi) / 360
    theta2 = longitude2 * (2 * pi) / 360
    phi1 = (90 - latitude1) * (2 * pi) / 360
    phi2 = (90 - latitude2) * (2 * pi) / 360
    distance = acos(sin(phi1) * sin(phi2) * cos(abs(theta1 - theta2)) + cos(phi1) * cos(phi2)) * radius_earth
    return floor(distance)

## Calculate distances between each airport pair

The time complexity is O(n^4). This function will use the coordinates of each airport to calculate the distances between each airport from a list of possible legs from the routes dictionary. 

In [9]:
def leg_distance_calculator(listx, airport_list):
    """Takes list of route and aircraft required. Also takes in dictionary of airport objects. Creates permutation
    list of airports. Using the objects passed, it finds the distance between each airport and saves the leg
    and distance as a key and value in a dictionary"""
    airport_distances = {}
    permlist = allPerms(listx)

    for perms in permlist:
        for i in range(0, len(perms) - 1):
            for j in airport_list:
                if j == perms[i]:
                    airport1 = airport_list[perms[i]]
                    for k in airport_list:
                        if k == perms[i + 1]:
                            airport2 = airport_list[perms[i + 1]]
                    distance = distanceBetweenAirports(float(airport1.longitude), float(airport1.latitude),
                                                       float(airport2.longitude), float(airport2.latitude))
                    if distance != 0:
                        airport_distances['_'.join([airport1.airport_code, airport2.airport_code])] = distance

    return airport_distances


## Costing

## Calculating the Cost of each leg

Takes in leg dictionary and airport objects. For each leg, it takes the departure airport and gets the local currency conversion rate. It then multiplies it by the distance to get the cost of each leg. Returns a dictionary with the cost of each leg. Time complexity is O(n^2)

In [10]:
def findLegCosts(leg_distance_dict, airport_object_dict):
    costDict = {} #create dictionary to store leg costs
    for i in leg_distance_dict:
        try:
            myKey = i[:3] #strip out first three letters, which will be home airport's code (ex: DUB)
        except IndexError:
            print("There's an error in how the dictionary is built! Each key of the leg distance dict should have 'DUB_LHR' style formatting. The program will now fail.")
        for j in airport_object_dict:
            if myKey == j: #if key is in dictionary
                cost = round(float(airport_object_dict[j].exchange_rate) * float(leg_distance_dict[i]), 2) # multiply exchange rate and distance of airport object, which is value for key
                costDict[i] = cost #add to new dictionary, with same key as leg_distance_dict
    return costDict


## Calculating the Cost of each Route

Time complexity is O(n^2). This returns the total cost of each route - using the cost of each leg. Returned in a dictionary with: Key: route(tuple) & Values: Total cost"""

In [11]:
def findRouteCost(list_of_route_lists, costDict):
    routeCostDict = {} # create dictionary to store total route costs
    x = 0
    while x < len(list_of_route_lists): #loop through outer lists
        i = 0
        cost = 0
        while i < (len(list_of_route_lists[x]) - 1): # loop through inner list
            myKey = str(list_of_route_lists[x][i]) + "_" + str(list_of_route_lists[x][i + 1]) # connect first and second leg of route in format "DUB_LHR"
            cost += costDict[myKey] #find cost in dictionary of costs for each leg
            cost = round(cost, 2) #round to nearest 2nd decimal place
            i += 1
        routeCostDict[tuple(list_of_route_lists[x])] = cost # add tuple route list as key, value being cost
        x += 1
    return routeCostDict 

## Capability of Aircraft

The time complexity is O(n^2), this will make sure that the plane's maximum range is not less than the distance of each leg in the route. If it is, that route will be removed from the dictionary of routes.

In [12]:
def checkAircraftAllowed(dictAirplane, distanceDict, input_list):
    """Checks that the aircraft being used can do the route. Returns the routes that are
    only possible with the aircraft"""
    planeToFly = input_list[5]
    planeRange = dictAirplane[planeToFly].max_capacity
    distanceDict_copy = distanceDict.copy()
    for j in distanceDict_copy:
        if distanceDict_copy[j] < int(planeRange):
            distanceDict.pop(j)
    finalRouteDict_copy = finalRouteDict.copy()
    for i in finalRouteDict_copy:
        toRemove = False
        for j in distanceDict:
            x = 0
            while x < len(i) - 1:
                if str(i[x] + "_" + i[x + 1]) == j:
                    toRemove = True
                x += 1
        if toRemove:
            finalRouteDict.pop(i)

## Caching

## Caching all the routes

This method has a constant time complexity. It simply stores each route into a dictionary with the first airport as the key, and the rest of the airports as the value.

In [13]:
def cacheRoutes(airportsInRoute,plane,cost,dictCache):
    ''''''
    #This first line will take the first aiport and store it as a variable
    firstAirport = airportsInRoute[0]
    #The next line gets the other remaining airports
    middleAirports = airportsInRoute[1:-1]
    #These next two lines append the plane and cost, so the middleAirports list contains the airports in the route
    #not including the first one, the plane used on the route, and the cost. 
    middleAirports.append(plane)
    middleAirports.append(str(cost))
    #This adds the aforementioned list to a dictionary with the first airport as the key
    dictCache[firstAirport] = middleAirports

## Check if a route exists in the cache already

This function, with a time complexity of O(n), will check if a list of airports and a plane has already been passed into the program, and if it has, will skip doing all the calculations and used the cache result from the last calculation to avoid doing another round of processing

In [14]:
def checkCache(airportList, myCache):
    ''''''
    #Go into this section if the cache dictionary actually has something in it
    if len(myCache) != 0:
        for i in myCache:
            #check if the first airport in the route matches any keys in the cache dictionary
            if airportList[0] == i:
                #If they do match, now convert the airport list you passed in (which will be the input the user
                #has provided) to a set, and then convert the cache's airports to a set and compare the two. This
                #is because the set ADT allows for comparison even if the order of the two sets do not match.
                #We also check if the planes of both routes match.
                if set(airportList[1:-1]) == set(myCache[i][0:-2]) and airportList[-1].rstrip() == myCache[i][-2]:
                    #If these conditions are met, we build a list with the first airport at the beginning, the middle
                    #airports as the next ones, and then the first one at the end again since the plane returns. We
                    #also append the cost to the list. The main code block will handle parsing the list to display results.
                    myReturn = [str(i)]
                    myReturn.extend(myCache[i][0:4])
                    myReturn.append(str(i))
                    myReturn.append(str(myCache[i][5]))
                    return myReturn
                else:
                    return False
    return False

## Analysis

This block of code will put all the classes and methods together. 

In [17]:
cacheDict = {}
print("Please enter the name of the file containing the routes: ")
file_name = input("> ")
while not os.path.isfile(file_name):
    print("I am sorry. I do not believe that file exists. Remember, it is case sensitive. Please try again.")
    print("=" * 75)
    print("")
    file_name = input("> ")

print(f"Opening {file_name}....")
datamatrix = read_file(file_name)
i = 0

if os.path.exists('finalResults.csv'):
    os.remove('finalResults.csv')
else:
    pass

while i < len(datamatrix):
    inputList = datamatrix[i]  # sample list passed
    cacheCheck = checkCache(inputList,cacheDict)
    if cacheCheck != False:
        resultList = [cacheCheck[0:-1]]
        resultList.append(cacheCheck[-1])
        with open('finalResults.csv', 'a') as csvFile:
            writer = csv.writer(csvFile)
            writer.writerow(resultList)
        
        csvFile.close()
        i += 1
        continue
    else:
        pass
    inputList[-1] = inputList[-1].rstrip()
    dictOfAircrafts = buildAircraft(inputList[-1])  # Creates aircraft objects
    if not dictOfAircrafts:
        print("The airplane specified does not exist, skipping this route.")
        i += 1
        continue
    airport_objects_dict = buildAirports(inputList)  # creates the objects for each airport
    if not airport_objects_dict:
        print("One of the airports specified does not exist, skipping this route.")
        i += 1
        continue

    # create all the possible routes
    all_routes_list = allPerms(inputList)

    # Finds distances and costs of each leg
    dict_routes_distances = leg_distance_calculator(inputList, airport_objects_dict)
    leg_costs = findLegCosts(dict_routes_distances, airport_objects_dict)

    # Finds total route cost
    finalRouteDict = findRouteCost(all_routes_list, leg_costs)

    # Removes distances that the aircraft cannot do
    checkAircraftAllowed(dictOfAircrafts, dict_routes_distances, inputList)
    if (len(finalRouteDict)) == 0:
        result = "This route is not possible with the " + str(inputList[-1]) + ". Try another plane. "
        resultList = [result]
        with open('finalResults.csv', 'a') as csvFile:
            writer = csv.writer(csvFile)
            writer.writerow(resultList)
        
        csvFile.close()
        i += 1
        continue
    cheapestRoute = min(finalRouteDict, key=finalRouteDict.get)
    cost = finalRouteDict[cheapestRoute]
    resultList = [cheapestRoute]
    resultList.append(cost)
    with open('finalResults.csv', 'a') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerow(resultList)
        
    csvFile.close()
    cacheRoutes(list(cheapestRoute), inputList[-1].rstrip(), cost, cacheDict)
    
    i += 1

print("Finished! The output is stored as finalResults in your working directory.")

Please enter the name of the file containing the routes: 
> test.csv
Opening test.csv....
Finished! The output is stored as finalResults in your working directory.


In [16]:
import unittest
import math
from itertools import permutations


class TestMethods(unittest.TestCase):

    def test_build_aircraft(self):
        aircraft_dict = buildAircraft('777')
        self.assertIsNotNone(aircraft_dict)
    
    def test_aircraft_dict(self):
        aircraft_dict = buildAircraft('777')
        self.assertIsInstance(aircraft_dict, dict)
    
    def test_aircraft_object(self):
        aircraft_dict = buildAircraft('777')
        for key,values in aircraft_dict.items():
            self.assertIsInstance(values, Aircraft)
    
    def test_aircraft_private(self):
        aircraft_dict = buildAircraft('777')
        for key,values in aircraft_dict.items():
            self.assertNotIsInstance(values._max_capacity, Aircraft)
            self.assertNotIsInstance(values._model, Aircraft)
            self.assertNotIsInstance(values._current_capacity, Aircraft)
            
    def test_permutation_number(self):
        inputlist1 = ['CDG', 'SYD', 'LIN', 'CPH']
        factorialnum = math.factorial(len(inputlist1))
        perms1 = list(permutations(inputlist1))
        list1 = []
        for i in perms1:
            i = list(i)
            list1.append(i)
        perms2 = ['CDG', 'CDG', 'CDG', 'CDG']
        self.assertTrue(len(permutation(inputlist1)) == factorialnum)
        self.assertTrue(permutation(inputlist1)==list1)
        self.assertFalse(permutation(inputlist1)==perms2)

    def test_departure_arrival(self):
        inputList = ['DUB', 'LHR', 'MOS', 'HEL', 'CPH']
        permlist = allPerms(inputList)
        i = 0
        while i < len(inputList):
            self.assertTrue(inputList[0] == permlist[i][0] and inputList[0] == permlist[i][4])
            i+=1

    def test_airport_distance(self):
        inputList = ['DUB', 'LHR', 777]
        airport_objects_dict = buildAirports(inputList)
        all_routes_list = allPerms(inputList) 
        dict_routes_distances = leg_distance_calculator(inputList, airport_objects_dict)
        distance = distanceBetweenAirports(53.421333, -6.270075, 51.4775, -0.461389)
        for i in dict_routes_distances:
            if dict_routes_distances[i] == 448:
                self.assertTrue(leg_distance_calculator(inputList, airport_objects_dict))
             
    def test_build_airports(self):
        inputList = ['DUB', 'LHR', 'SYD', 'JFK', 'AAL', '777']
        airport_objects_dict = buildAirports(inputList)
        self.assertIsNotNone(airport_objects_dict)
    
    def test_airports_dict(self):
        inputList = ['DUB', 'LHR', 'SYD', 'JFK', 'AAL', '777']
        airport_objects_dict = buildAirports(inputList)
        self.assertIsInstance(airport_objects_dict, dict)
    
    def test_airport_object(self):
        inputList = ['DUB', 'LHR', 'SYD', 'JFK', 'AAL', '777']
        airport_objects_dict = buildAirports(inputList)
        for key,values in airport_objects_dict.items():
            self.assertIsInstance(values, Airport)
    
    def test_airport_private(self):
        inputList = ['DUB', 'LHR', 'SYD', 'JFK', 'AAL', '777']
        airport_objects_dict = buildAirports(inputList)
        for key,values in airport_objects_dict.items():
            self.assertNotIsInstance(values._longitude, Airport)
            self.assertNotIsInstance(values._latitude, Airport)
            self.assertNotIsInstance(values._exchange_rate, Airport)

    def test_caching_check(self):
        cacheDict = {"SNN": ["ORK", "SIN", "CDG", "MAN", "A330", "19773.1"]}
        self.assertNotEqual(checkCache(["SNN", "ORK", "MAN", "SIN", "CDG", "A330"], cacheDict), False)
    
    def test_caching_check2(self):
        cacheDict = {"SNN": ["ORK", "SIN", "CDG", "MAN", "777", "19773.1"]}
        self.assertFalse(checkCache(["SNN", "ORK", "MAN", "SIN", "CDG", "A330" ], cacheDict))
        
    def test_cost_routes_length(self):
        inputList = ['LHR', 'SFO', 'HEL', 'JFK', 'SYD', '777']
        airport_objects_dict = buildAirports(inputList)  
        all_routes_list = allPerms(inputList)
        dict_routes_distances = leg_distance_calculator(inputList, airport_objects_dict)
        leg_costs = findLegCosts(dict_routes_distances, airport_objects_dict)
        self.assertEqual(len(leg_costs), 20)
        
    def test_cost_routes_calc(self):
        inputList = ['LHR', 'SFO', 'HEL', 'JFK', 'SYD', '777']
        airport_objects_dict = buildAirports(inputList)  
        all_routes_list = allPerms(inputList)
        dict_routes_distances = leg_distance_calculator(inputList, airport_objects_dict)
        dist = dict_routes_distances['LHR_JFK']
        dist = round(dist*1.4029,2)
        leg_costs = findLegCosts(dict_routes_distances, airport_objects_dict)
        self.assertEqual(leg_costs['LHR_JFK'], dist)
        
    def test_total_routes_length(self):
        inputList = ['LHR', 'SFO', 'HEL', 'JFK', 'SYD', '777']
        airport_objects_dict = buildAirports(inputList)  
        all_routes_list = allPerms(inputList)
        dict_routes_distances = leg_distance_calculator(inputList, airport_objects_dict)
        dist = dict_routes_distances['LHR_JFK']
        dist = round(dist*1.4029,2)
        leg_costs = findLegCosts(dict_routes_distances, airport_objects_dict)
        finalRouteDict = findRouteCost(all_routes_list, leg_costs)
        self.assertEqual(len(finalRouteDict), 24)
        
    def test_total_routes_cost(self):
        inputList = ['LHR', 'SFO', 'HEL', 'JFK', 'SYD', '777']
        airport_objects_dict = buildAirports(inputList)  
        all_routes_list = allPerms(inputList)
        dict_routes_distances = leg_distance_calculator(inputList, airport_objects_dict)
        dist = dict_routes_distances['LHR_JFK']
        dist = round(dist*1.4029,2)
        leg_costs = findLegCosts(dict_routes_distances, airport_objects_dict)
        finalRouteDict = findRouteCost(all_routes_list, leg_costs)
        cost = leg_costs['LHR_SFO'] + leg_costs['SFO_HEL'] + leg_costs['HEL_JFK'] + leg_costs['JFK_SYD'] + leg_costs['SYD_LHR']
        cost = round(cost,2)
        self.assertEqual(finalRouteDict['LHR', 'SFO', 'HEL', 'JFK', 'SYD', 'LHR'], cost)


if __name__ == '__main__':
     unittest.main(argv=['ignored', '-v'], exit=False)

test_aircraft_dict (__main__.TestMethods) ... ok
test_aircraft_object (__main__.TestMethods) ... ok
test_aircraft_private (__main__.TestMethods) ... ok
test_airport_distance (__main__.TestMethods) ... ok
test_airport_object (__main__.TestMethods) ... ok
test_airport_private (__main__.TestMethods) ... ok
test_airports_dict (__main__.TestMethods) ... ok
test_build_aircraft (__main__.TestMethods) ... ok
test_build_airports (__main__.TestMethods) ... ok
test_caching_check (__main__.TestMethods) ... ok
test_caching_check2 (__main__.TestMethods) ... ok
test_cost_routes_calc (__main__.TestMethods) ... ok
test_cost_routes_length (__main__.TestMethods) ... ok
test_departure_arrival (__main__.TestMethods) ... ok
test_permutation_number (__main__.TestMethods) ... ok
test_total_routes_cost (__main__.TestMethods) ... ok
test_total_routes_length (__main__.TestMethods) ... ok

----------------------------------------------------------------------
Ran 17 tests in 0.179s

OK
